# Forecasting with AR Models

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

## Sim Data

Simulating a sine way with normally distributed error at each step. AR(1) model is not particularly appropriate for this, but this is just an illustration.

In [ ]:
np.random.seed(111)

# Simulation Parameters
N = 100 # Number of points to sim
mu = 0 # Random error mean
sd = 0.2 # Random error std
T = 4 # Period

# Sim Data
x = np.linspace(0, 5 * np.pi, N)
y = np.sin(2 * np.pi / T * x) + np.random.normal(mu, sd, N)

# Cross val parameters
h2 = int(0.5 * N) # Time step when training ends, forecast begins

In [ ]:
# Plot the result
plt.plot(x, y)
plt.xlabel('x')
plt.ylabel('y')
plt.axvline(x=x[h2], color='k', linestyle="dashed", label='Forecast Start')
# plt.legend()
plt.show()

## Fit AR Model

We use `statsmodels` package and reproduce with typical linear regression from `sklearn`. 

*NOTE:* this software **does not** allow for weighted least squares nor custom loss functions.

**Confirm:**
* Does `statsmodels` use Yule-Walker? [Documentation](https://www.statsmodels.org/dev/generated/statsmodels.tsa.ar_model.AutoReg.html) says: "Estimate an AR-X model using Conditional Maximum Likelihood (OLS)."
* Does `sklearn` `LinearRegression` use least-squares? [Documentaton](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) says: "From the implementation point of view, this is just plain Ordinary Least Squares (scipy.linalg.lstsq) or Non Negative Least Squares (scipy.optimize.nnls) wrapped as a predictor object."

### Using Existing Software Tools

`statsmodels.tsa.ar_model`

In [ ]:
train = y[0:h2]
ar = AutoReg(train, lags=1).fit()

In [ ]:
ar.summary()

In [ ]:
fitted_ar = ar.predict(start=1, end=len(train)-1)

In [ ]:
plt.plot(x[:h2], y[:h2], label="True Data")  # Full training data
plt.plot(x[1:h2], fitted_ar, label="AR(1) Fitted", linestyle="dashed")  # Predictions start at y_1
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

In [ ]:
r2_score(fitted_ar, train[1:])

### Manual Implementation

`sklearn.linear_model.LinearRegression`

*NOTE:* this software allows for weighted least squares nor custom loss functions.

From the response data $y_t$ in the training period, get the lag-1 time series $y_{t-1}$ and fit in a linear regression with intercept:

$$
y_t = \mu + \theta y_{t-1} +\epsilon
$$

Note: fitted values will start at time $t=1$ rather than time $t=0$ (counting from zero bc of python)

In [ ]:
ylag = train[:-1].reshape(-1, 1)

In [ ]:
lm_ar = LinearRegression()
lm_ar.fit(X=ylag, y=train[1:])

In [ ]:
lm_ar.coef_

In [ ]:
lm_ar.intercept_

In [ ]:
fitted_lm = lm_ar.predict(train[:-1].reshape(-1, 1))

In [ ]:
np.max(np.abs(fitted_lm - fitted_ar))

## Forecasting 

Predicting with model in future, aka **extrapolation in time**.

### The Wrong Way

Following the method for fitted the data, we take the time series $y_t$ in the forecast/test period and get the lag-1 version of it $y_{t-1}$, then predict with fitted coefs. Let $\hat y_t$ be predicted model output at time $t$, $\hat \mu$ be the estimated intercept, and $\hat \theta$ be the estimated autoregressive coefficient.

$$
\hat y_t = \hat \mu + \hat \theta y_{t-1}
$$

**Question: what is wrong with this approach to forecasting?**

In [ ]:
test = y[h2:]
ylag2 = test[:-1].reshape(-1, 1)

In [ ]:
preds0 = lm_ar.predict(train[:-1].reshape(-1, 1))

In [ ]:
r2_score(preds0, test[1:])

In [ ]:
plt.plot(x, y)
plt.plot(x[1:h2], fitted_lm, label="Fitted Values")
plt.plot(x[(h2+1):], preds0, label="'Forecasted' Values")
plt.xlabel('x')
plt.ylabel('y')
plt.axvline(x=x[h2], color='k', linestyle="dashed", label='Forecast Start')
plt.legend()
plt.show()

### The Right Way

**Explanation of Error Above:** instead of *forecasting* with the model, the real observed time series was used to generate a predicted time series. This information would not be available to you in a real-world context. What we showed above is something in between fitted values, where we predict observations that were used to fit model parameters, and true predictions, where we predict observations that are not used to inform the model parameters in any way. This is not the right way to perform forecasting, where the goal is to extrapolate in time to predict values that have not been observed yet.

**How it Should be Done:**

Iterative scheme where model output at time $t$ used to predict $t+1$:
* Starting at first time of forecast period, predict $\hat y_t$ using estimated coefs as above
* Predict: $\hat y_{t+1} = \hat \mu + \hat \theta \hat y_{t}$

Note: When forecasting with simple ARMA models with no external inputs, as is the case here, there is a tendency for the forecasts to decay to the estimated mean $\hat \mu$

In [ ]:
preds_ar = ar.predict(start=h2, end=len(y)-1)

In [ ]:
plt.plot(x, y)
plt.plot(x[1:h2], fitted_ar, label="Fitted Values")
plt.plot(x[h2:], preds_ar, label="Forecasted Values")
plt.xlabel('x')
plt.ylabel('y')
plt.axvline(x=x[h2], color='k', linestyle="dashed", label='Forecast Start')
plt.legend()
plt.show()

In [ ]:
preds_lm = []
pred_t_minus_1 = lm_ar.predict(train[-1].reshape(-1, 1))[0]
preds_lm.append(pred_t_minus_1)
for t in range(h2, len(y)-1):
    pred_t_minus_1 = lm_ar.predict(np.array([[pred_t_minus_1]]))[0] 
    preds_lm.append(pred_t_minus_1)

preds_lm = np.array(preds_lm)

In [ ]:
np.max(np.abs(preds_lm - preds_ar))

In [ ]:
plt.plot(x, y)
plt.plot(x[1:h2], fitted_lm, label = "Fitted")
plt.plot(x[h2:], preds_lm, label = "Forecast")
plt.legend()